In [ ]:
def get_namespace_errors(namespace):
    from llama_stack_client import LlamaStackClient, Agent
    from rich.pretty import pprint
    import uuid
    from pydantic import BaseModel


    # Uncomment to run on a hosted Llama Stack server
    client = LlamaStackClient(base_url="http://localhost:8321")

    MODEL_ID = "llama-4-scout-17b-16e-w4a16" #"llama-3-2-3b" #"r1-qwen-14b-w4a16" # "llama-4-scout-17b-16e-w4a16"

    base_agent_config = dict(
        model=MODEL_ID,
        instructions="You are an expert OpenShift administrator. Analyze the namespace '{namespace}' for errors and issues.",
        sampling_params={
            "strategy": {"type": "top_p", "temperature": 1.0, "top_p": 0.9},
        },
        tools=[
            {
                "name": "mcp::openshift",
                "args": {"namespace": namespace}
            }
        ],
    )

    vanilla_agent_config = {
        **base_agent_config,
        "instructions": """
        You are an expert OpenShift administrator. Analyze the namespace '{namespace}' for errors and issues.
        """,
    }

    vanilla_agent = Agent(client, **vanilla_agent_config)
    prompt_chaining_session_id = vanilla_agent.create_session(session_name=f"vanilla_agent_{uuid.uuid4()}")

    prompts = [
        """
        Analyze the namespace '{namespace}' by:
                1. Listing all pods 
                2. get pod logs for the pods listed above
                2. Checking events for any errors or warnings
                3. Providing initial assessment""",
        """Format the output as JSON:
                {
                    "namespace": "{namespace}",
                    "analysis_results": [
                        {
                            "Pod": "[pod name]",
                            "Error Title": "[error type/reason]",
                            "Error Timestamp": "[timestamp]"
                            "Error Description": "[detailed error message]"
                        }
                    ]
                }
                """,
    ]

    for i, prompt in enumerate(prompts):    
        response = vanilla_agent.create_turn(
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
            session_id=prompt_chaining_session_id,
            stream=False,
        )
        print("************ Turn: ", i, "****************")
        print(response.output_message.content)
        print("\n")
    return response.output_message.content

In [ ]:
# Simple standalone function to create Jira incidents
def create_jira_incident(incident_details, config=None):
    """
    Create a Jira incident using ReActAgent with automatic tool execution.
    This is a standalone function that creates its own agent and session.
    
    Args:
        incident_details (dict): Dictionary containing incident details with keys:
            - project_key: Jira project key (e.g., 'KAN')
            - summary: Issue summary/title
            - description: Detailed description of the incident
            - issue_type: Type of issue (e.g., 'Incident', 'Bug', 'Task')
        config (dict, optional): Configuration overrides with keys:
            - base_url: LlamaStack server URL (default: 'http://localhost:8321')
            - model_id: Model ID (default: 'r1-qwen-14b-w4a16')
            - temperature: Temperature for sampling (default: 0.0)
            - max_tokens: Maximum tokens (default: 4096)
            - max_infer_iters: Max inference iterations (default: 10)
    
    Returns:
        dict: Response from the agent with execution details
    """
    
    # Import required modules
    import os
    from dotenv import load_dotenv
    from llama_stack_client import LlamaStackClient
    from llama_stack_client.lib.agents.react.agent import ReActAgent
    from llama_stack_client.lib.agents.react.tool_parser import ReActOutput
    
    # Load configuration
    load_dotenv('config.env')
    
    # Set default configuration
    default_config = {
        'base_url': os.environ.get('LLAMA_STACK_URL', 'http://localhost:8321'),
        'model_id': os.environ.get('LLM_MODEL_ID', 'r1-qwen-14b-w4a16'),
        'temperature': float(os.environ.get("TEMPERATURE", 0.0)),
        'max_tokens': int(os.environ.get("MAX_TOKENS", 4096)),
        'max_infer_iters': int(os.environ.get("MAX_INFER_ITERATIONS", 10))
    }
    
    # Override with provided config
    if config:
        default_config.update(config)
    
    # Validate required fields
    required_fields = ['project_key', 'summary', 'description', 'issue_type']
    for field in required_fields:
        if field not in incident_details:
            raise ValueError(f"Missing required field: {field}")
    
    print("🚀 Initializing ReActAgent for Jira incident creation...")
    print(f"🔗 Connecting to: {default_config['base_url']}")
    print(f"🤖 Using model: {default_config['model_id']}")
    print(f"🔄 Max inference iterations: {default_config['max_infer_iters']}")
    
    try:
        # Create client
        client = LlamaStackClient(base_url=default_config['base_url'])
        print("✅ Client created")
        
        # Sampling parameters
        sampling_params = {
            "strategy": {"type": "greedy"},
            "max_tokens": default_config['max_tokens'],
            "temperature": default_config['temperature'],
        }
        
        # Create agent with AUTO tool execution
        agent = ReActAgent(
            client=client,
            model=default_config['model_id'],
            tools=["mcp::atlassian"],  # Specify tool groups
            response_format={
                "type": "json_schema",
                "json_schema": ReActOutput.model_json_schema(),
            },
            sampling_params=sampling_params,
            max_infer_iters=default_config['max_infer_iters']  # This is key for auto execution!
        )
        print("✅ ReActAgent created with AUTO tool execution")
        
        # Create session
        session_id = agent.create_session("jira-incident-creation")
        print(f"✅ Session created: {session_id}")
        
    except Exception as e:
        print(f"❌ Failed to initialize agent: {e}")
        import traceback
        traceback.print_exc()
        return {
            "success": False,
            "error": f"Agent initialization failed: {str(e)}",
            "traceback": traceback.format_exc()
        }
    
    # Build the simple prompt for creating the incident
    prompt = f"Create a Jira {incident_details['issue_type']} in the {incident_details['project_key']} project. Summary: {incident_details['summary']}. Description: {incident_details['description']}."
    
    print(f"\n🔧 Creating Jira incident with details:")
    print(f"   Project: {incident_details['project_key']}")
    print(f"   Type: {incident_details['issue_type']}")
    print(f"   Summary: {incident_details['summary']}")
    print(f"   Description: {incident_details['description']}")
    print(f"\n📝 Generated prompt: {prompt}")
    print("\n" + "="*60)
    
    try:
        # Create turn with the incident details
        response = agent.create_turn(
            messages=[{"role": "user", "content": prompt}],
            session_id=session_id,
            stream=False  # Non-streaming for auto execution
        )
        
        print("\n🤖 ReActAgent response:")
        print(f"Response: {response.output_message.content}")
        
        # Display execution steps
        print(f"\n🔍 Execution steps: {len(response.steps)}")
        for i, step in enumerate(response.steps):
            print(f"Step {i+1}: {step.step_type}")
            if hasattr(step, 'api_model_response') and step.api_model_response:
                content = step.api_model_response.content
                if len(content) > 200:
                    content = content[:200] + "..."
                print(f"  Content: {content}")
        
        return {
            "success": True,
            "response": response,
            "steps_count": len(response.steps),
            "output": response.output_message.content,
            "session_id": session_id,
            "agent_config": default_config
        }
        
    except Exception as e:
        print(f"❌ Failed to create Jira incident: {e}")
        import traceback
        traceback.print_exc()
        return {
            "success": False,
            "error": str(e),
            "traceback": traceback.format_exc(),
            "session_id": session_id,
            "agent_config": default_config
        }

print("✅ Simple standalone function 'create_jira_incident' defined successfully!")
print("   Usage: create_jira_incident(incident_details, config=None)")
print("   - Only requires: project_key, summary, description, issue_type")
print("   - No need to create agent or session manually")
print("   - Automatically handles all initialization")

In [ ]:
# Test both functions together - UPDATED VERSION
print("🧪 Testing both functions together")
print("="*50)

# Step 1: Get errors from namespace using our function
print("\n📊 STEP 1: Getting errors from namespace using get_namespace_errors()...")
namespace = "oom-test"
errors_result_raw = get_namespace_errors(namespace)
print("Raw result:")
print(errors_result_raw)

# Parse the JSON string into a dictionary
import json
try:
    errors_result = json.loads(errors_result_raw)
    print("\n✅ Successfully parsed JSON result")
except json.JSONDecodeError as e:
    print(f"\n❌ Failed to parse JSON: {e}")
    print("Raw content:", repr(errors_result_raw))
    errors_result = {"namespace": namespace, "analysis_results": []}

print(f"\n📋 Error analysis results from get_namespace_errors():")
print(f"   Namespace: {errors_result.get('namespace', 'Unknown')}")
print(f"   Errors found: {len(errors_result.get('analysis_results', []))}")

# Display errors found
if errors_result.get('analysis_results'):
    for i, error in enumerate(errors_result['analysis_results']):
        print(f"   Error {i+1}:")
        print(f"     Pod: {error.get('Pod', 'Unknown')}")
        print(f"     Type: {error.get('Error Title', 'Unknown')}")
        print(f"     Time: {error.get('Error Timestamp', 'Unknown')}")
        print(f"     Description: {error.get('Error Description', 'No description')[:100]}...")
        print()

# Step 2: Create Jira incidents for each error using create_jira_incident
if errors_result.get('analysis_results'):
    print(f"\n🎫 STEP 2: Creating Jira incidents for {len(errors_result['analysis_results'])} errors...")
    
    jira_results = []
    for i, error in enumerate(errors_result['analysis_results']):
        print(f"\n--- Creating Jira incident {i+1}/{len(errors_result['analysis_results'])} ---")
        # Map the error data to the expected format for create_jira_incident
        incident_details = {
            'project_key': 'KAN',
            'summary': f"OpenShift Error - {error.get('Error Title', 'Unknown Error')} in Pod {error.get('Pod', 'Unknown')}",
            'description': f"""Pod: {error.get('Pod', 'Unknown')}
Error Type: {error.get('Error Title', 'Unknown')}
Timestamp: {error.get('Error Timestamp', 'Unknown')}
Error Details: {error.get('Error Description', 'No description')}

Priority: High
Labels: openshift, pod-error""",
            'issue_type': 'Incident'
        }
        jira_result = create_jira_incident(incident_details)
        jira_results.append(jira_result)
        
        if jira_result['success']:
            print(f"✅ Jira incident {i+1} created successfully")
        else:
            print(f"❌ Jira incident {i+1} failed: {jira_result.get('error', 'Unknown error')}")
    
    # Summary
    successful_incidents = sum(1 for result in jira_results if result['success'])
    print(f"\n🎉 FINAL SUMMARY:")
    print(f"   Namespace analyzed: {namespace}")
    print(f"   Errors found: {len(errors_result['analysis_results'])}")
    print(f"   Jira incidents created: {successful_incidents}/{len(errors_result['analysis_results'])}")
    
else:
    print("\nℹ️  No errors found, skipping Jira incident creation")

print("\n✅ Test completed!")


In [ ]:
# Function to search Confluence pages in the 'ocp' space
def search_confluence_pages(search_query, space_key="ocp", config=None):
    """
    Search Confluence pages in the specified space using Agent with tool execution.
    This is a standalone function that creates its own agent and session.
    
    Args:
        search_query (str): The search query to find pages in Confluence
        space_key (str): The Confluence space key to search in (default: 'ocp')
        config (dict, optional): Configuration overrides with keys:
            - base_url: LlamaStack server URL (default: 'http://localhost:8321')
            - model_id: Model ID (default: 'r1-qwen-14b-w4a16')
            - temperature: Temperature for sampling (default: 0.0)
            - max_tokens: Maximum tokens (default: 4096)
    
    Returns:
        dict: Response from the agent with search results and execution details
    """
    
    # Import required modules
    import os
    import uuid
    from dotenv import load_dotenv
    from llama_stack_client import LlamaStackClient, Agent
    
    # Load configuration
    load_dotenv('config.env')
    
    # Set default configuration
    default_config = {
        'base_url': os.environ.get('LLAMA_STACK_URL', 'http://localhost:8321'),
        'model_id': os.environ.get('LLM_MODEL_ID', 'r1-qwen-14b-w4a16'),
        'temperature': float(os.environ.get("TEMPERATURE", 0.0)),
        'max_tokens': int(os.environ.get("MAX_TOKENS", 4096))
    }
    
    # Override with provided config
    if config:
        default_config.update(config)
    
    # Validate required fields
    if not search_query or not search_query.strip():
        raise ValueError("Search query cannot be empty")
    
    print("🔍 Initializing Agent for Confluence page search...")
    print(f"🔗 Connecting to: {default_config['base_url']}")
    print(f"🤖 Using model: {default_config['model_id']}")
    print(f"📝 Search query: '{search_query}'")
    print(f"🏢 Space key: '{space_key}'")
    
    try:
        # Create client
        client = LlamaStackClient(base_url=default_config['base_url'])
        print("✅ Client created")
        
        # Agent configuration with explicit tool execution
        agent_config = {
            "model": default_config['model_id'],
            "instructions": f"""You are an expert Confluence administrator. When asked to search for pages, you MUST use the available Confluence search tools to find actual pages and return the results.

IMPORTANT: 
- Always execute the confluence_search tool when asked to search
- Return the actual search results, not just the tool call
- Format results clearly with page titles, URLs, and descriptions
- If no results are found, clearly state that no pages were found""",
            "sampling_params": {
                "strategy": {"type": "greedy"},
                "max_tokens": default_config['max_tokens'],
                "temperature": default_config['temperature'],
            },
            "tools": [
                {
                    "name": "mcp::atlassian",
                    "args": {"space_key": space_key}
                }
            ],
        }
        
        # Create agent
        agent = Agent(client, **agent_config)
        print("✅ Agent created with Confluence tools")
        
        # Create session
        session_id = agent.create_session(session_name=f"confluence_search_{uuid.uuid4()}")
        print(f"✅ Session created: {session_id}")
        
    except Exception as e:
        print(f"❌ Failed to initialize agent: {e}")
        import traceback
        traceback.print_exc()
        return {
            "success": False,
            "error": f"Agent initialization failed: {str(e)}",
            "traceback": traceback.format_exc()
        }
    
    # Build a more explicit search prompt
    prompt = f"""Please search for Confluence pages in the '{space_key}' space using the query: '{search_query}'.

Use the confluence_search tool to find pages and return the actual search results including:
- Page titles
- Page URLs  
- Brief descriptions of the content

Execute the search tool now and provide the results."""
    
    print(f"\n🔍 Searching Confluence pages with details:")
    print(f"   Space: {space_key}")
    print(f"   Query: {search_query}")
    print(f"\n📝 Generated prompt: {prompt}")
    print("\n" + "="*60)
    
    try:
        # Create turn with the search query
        response = agent.create_turn(
            messages=[{"role": "user", "content": prompt}],
            session_id=session_id,
            stream=False
        )
        
        print("\n🤖 Agent response:")
        print(f"Response: {response.output_message.content}")
        
        # Display execution steps if available
        if hasattr(response, 'steps') and response.steps:
            print(f"\n🔍 Execution steps: {len(response.steps)}")
            for i, step in enumerate(response.steps):
                print(f"Step {i+1}: {step.step_type}")
                if hasattr(step, 'api_model_response') and step.api_model_response:
                    content = step.api_model_response.content
                    if len(content) > 200:
                        content = content[:200] + "..."
                    print(f"  Content: {content}")
                # Check for tool execution results
                if hasattr(step, 'tool_execution') and step.tool_execution:
                    print(f"  Tool execution: {step.tool_execution}")
                if hasattr(step, 'tool_result') and step.tool_result:
                    print(f"  Tool result: {step.tool_result}")
        
        # Check if the response contains actual search results or just tool calls
        response_content = response.output_message.content
        if "confluence_search(" in response_content and "tool_execution" not in str(response.steps):
            print("\n⚠️  WARNING: Agent returned tool call instead of executing it")
            print("   This suggests the tool execution may not be working properly")
        
        return {
            "success": True,
            "response": response,
            "steps_count": len(response.steps) if hasattr(response, 'steps') else 0,
            "output": response.output_message.content,
            "session_id": session_id,
            "agent_config": default_config,
            "search_query": search_query,
            "space_key": space_key
        }
        
    except Exception as e:
        print(f"❌ Failed to search Confluence pages: {e}")
        import traceback
        traceback.print_exc()
        return {
            "success": False,
            "error": str(e),
            "traceback": traceback.format_exc(),
            "session_id": session_id,
            "agent_config": default_config,
            "search_query": search_query,
            "space_key": space_key
        }

print("✅ Confluence search function 'search_confluence_pages' defined successfully!")
print("   Usage: search_confluence_pages(search_query, space_key='ocp', config=None)")
print("   - search_query: The text to search for in Confluence pages")
print("   - space_key: The Confluence space to search in (default: 'ocp')")
print("   - No need to create agent or session manually")
print("   - Automatically handles all initialization")


✅ Confluence search function 'search_confluence_pages' defined successfully!
   Usage: search_confluence_pages(search_query, space_key='ocp', config=None)
   - search_query: The text to search for in Confluence pages
   - space_key: The Confluence space to search in (default: 'ocp')
   - No need to create agent or session manually
   - Automatically handles all initialization


In [ ]:
# Test the Confluence search function
print("🧪 Testing Confluence search function")
print("="*50)

# Test 1: Search for OpenShift related pages
print("\n📊 TEST 1: Searching for OOMKilled related pages in 'ocp' space...")
search_result_1 = search_confluence_pages("OOMKilled", space_key="ocp")
print(f"\n📋 Search result 1:")
print(f"   Success: {search_result_1['success']}")
if search_result_1['success']:
    print(f"   Steps executed: {search_result_1['steps_count']}")
    print(f"   Search query: {search_result_1['search_query']}")
    print(f"   Space key: {search_result_1['space_key']}")
    print(f"   Output preview: {search_result_1['output'][:200]}...")
else:
    print(f"   Error: {search_result_1.get('error', 'Unknown error')}")

print("\n" + "="*60)



🧪 Testing Confluence search function

📊 TEST 1: Searching for OOMKilled related pages in 'ocp' space...
🔍 Initializing Agent for Confluence page search...
🔗 Connecting to: http://localhost:8321
🤖 Using model: llama-4-scout-17b-16e-w4a16
📝 Search query: 'OOMKilled'
🏢 Space key: 'ocp'
✅ Client created
✅ Agent created with Confluence tools
✅ Session created: ff616e3a-27eb-44e2-b07e-e9f080931bd3

🔍 Searching Confluence pages with details:
   Space: ocp
   Query: OOMKilled

📝 Generated prompt: Please search for Confluence pages in the 'ocp' space using the query: 'OOMKilled'.

Use the confluence_search tool to find pages and return the actual search results including:
- Page titles
- Page URLs  
- Brief descriptions of the content

Execute the search tool now and provide the results.


🤖 Agent response:
Response: I can't directly execute the tool, but I can guide you through the process.

To search for Confluence pages in the 'ocp' space using the query: 'OOMKilled', you would use the `conf

In [ ]:
# Debug test - Simple Confluence search
print("🔧 DEBUG: Testing Confluence search with detailed logging")
print("="*60)

# Test with a simple query
search_result = search_confluence_pages("test", space_key="ocp")

print(f"\n📋 DEBUG RESULTS:")
print(f"Success: {search_result['success']}")
print(f"Steps count: {search_result['steps_count']}")

if search_result['success']:
    print(f"\n📝 Full response content:")
    print(f"{search_result['output']}")
    
    print(f"\n🔍 Response analysis:")
    if "confluence_search(" in search_result['output']:
        print("❌ Agent returned tool call instead of executing it")
    else:
        print("✅ Agent appears to have executed the tool")
        
    # Check if we have actual search results
    if "page" in search_result['output'].lower() or "result" in search_result['output'].lower():
        print("✅ Response contains search results")
    else:
        print("❌ Response does not contain search results")
else:
    print(f"❌ Error: {search_result.get('error', 'Unknown error')}")

print("\n" + "="*60)


In [18]:
# WORKING approach - Direct tool usage using the correct API
def search_confluence_pages_direct(search_query, space_key="ocp", config=None):
    """
    WORKING approach to search Confluence pages using direct tool calls.
    Uses the correct API methods from the working example.
    """
    
    # Import required modules
    import os
    from dotenv import load_dotenv
    from llama_stack_client import LlamaStackClient
    
    # Load configuration
    load_dotenv('config.env')
    
    # Set default configuration
    default_config = {
        'base_url': os.environ.get('LLAMA_STACK_URL', 'http://localhost:8321'),
        'model_id': os.environ.get('LLM_MODEL_ID', 'r1-qwen-14b-w4a16'),
    }
    
    # Override with provided config
    if config:
        default_config.update(config)
    
    print("🔍 WORKING Direct Confluence search approach...")
    print(f"🔗 Connecting to: {default_config['base_url']}")
    print(f"📝 Search query: '{search_query}'")
    print(f"🏢 Space key: '{space_key}'")
    
    try:
        # Create client
        client = LlamaStackClient(base_url=default_config['base_url'])
        print("✅ Client created")
        
        # Get available tools using the correct API
        tools = client.tools.list(toolgroup_id="mcp::atlassian")
        print(f"📋 Available Atlassian tools: {len(tools)}")
        
        # Look for Confluence tools
        confluence_tools = [t for t in tools if 'confluence' in str(t).lower()]
        print(f"🔍 Confluence tools found: {len(confluence_tools)}")
        
        for tool in confluence_tools:
            tool_name = getattr(tool, 'name', getattr(tool, 'identifier', str(tool)))
            print(f"   - {tool_name}")
        
        if not confluence_tools:
            print("❌ No Confluence tools found")
            # Show all available tools for debugging
            all_tools = [getattr(t, 'name', getattr(t, 'identifier', str(t))) for t in tools]
            print(f"Available tools: {all_tools[:10]}...")  # Show first 10
            return {
                "success": False,
                "error": "No Confluence tools available",
                "available_tools": all_tools
            }
        
        # Use confluence_search tool (if available)
        tool_name = "confluence_search"
        print(f"🔧 Using tool: {tool_name}")
        
        # Prepare tool arguments (following the working pattern)
        tool_kwargs = {
            "query": search_query,
            "limit": 10
        }
        
        if space_key:
            tool_kwargs["spaces_filter"] = space_key
        
        print(f"📝 Tool arguments: {tool_kwargs}")
        
        # Execute the tool using the working API
        result = client.tool_runtime.invoke_tool(
            tool_name=tool_name,
            kwargs=tool_kwargs
        )
        
        print(f"✅ Tool execution completed")
        print(f"📋 Result type: {type(result)}")
        print(f"📋 Result content: {result}")
        
        return {
            "success": True,
            "result": result,
            "tool_used": tool_name,
            "tool_args": tool_kwargs,
            "search_query": search_query,
            "space_key": space_key
        }
        
    except Exception as e:
        print(f"❌ Failed to search Confluence pages: {e}")
        import traceback
        traceback.print_exc()
        return {
            "success": False,
            "error": str(e),
            "traceback": traceback.format_exc()
        }

print("✅ WORKING Direct Confluence search function 'search_confluence_pages_direct' defined!")
print("   Uses the correct API methods from the working example")


✅ WORKING Direct Confluence search function 'search_confluence_pages_direct' defined!
   Uses the correct API methods from the working example


In [19]:
# Test the direct approach
print("🧪 Testing Direct Confluence search approach")
print("="*60)

# Test direct tool usage
direct_result = search_confluence_pages_direct("OpenShift", space_key="ocp")

print(f"\n📋 DIRECT APPROACH RESULTS:")
print(f"Success: {direct_result['success']}")

if direct_result['success']:
    print(f"Tool used: {direct_result.get('tool_used', 'Unknown')}")
    print(f"Search query: {direct_result.get('search_query', 'Unknown')}")
    print(f"Space key: {direct_result.get('space_key', 'Unknown')}")
    print(f"\n📝 Search results:")
    print(f"{direct_result.get('result', 'No results')}")
else:
    print(f"❌ Error: {direct_result.get('error', 'Unknown error')}")
    if 'available_tools' in direct_result:
        print(f"Available tools: {direct_result['available_tools']}")

print("\n" + "="*60)


🧪 Testing Direct Confluence search approach
🔍 WORKING Direct Confluence search approach...
🔗 Connecting to: http://localhost:8321
📝 Search query: 'OpenShift'
🏢 Space key: 'ocp'
✅ Client created
📋 Available Atlassian tools: 37
🔍 Confluence tools found: 10
   - confluence_search
   - confluence_get_page
   - confluence_get_page_children
   - confluence_get_comments
   - confluence_get_labels
   - confluence_add_label
   - confluence_create_page
   - confluence_update_page
   - confluence_delete_page
   - confluence_add_comment
🔧 Using tool: confluence_search
📝 Tool arguments: {'query': 'OpenShift', 'limit': 10, 'spaces_filter': 'ocp'}
✅ Tool execution completed
📋 Result type: <class 'llama_stack_client.types.tool_invocation_result.ToolInvocationResult'>
📋 Result content: ToolInvocationResult(content=[TextContentItem(text='[\n  {\n    "id": "262334",\n    "title": "The Pod Restarting Due to Out of Memory Error",\n    "type": "page",\n    "created": "",\n    "updated": "",\n    "url": "htt

In [20]:
# Simple working Confluence search using the exact pattern from working example
def search_confluence_simple(search_query, space_key="ocp"):
    """
    Simple Confluence search using the exact working pattern from simple-tool-test-working.ipynb
    """
    import os
    from dotenv import load_dotenv
    from llama_stack_client import LlamaStackClient
    
    # Load config
    load_dotenv('config.env')
    base_url = os.environ.get('LLAMA_STACK_URL', 'http://localhost:8321')
    print(f"🔗 Connecting to: {base_url}")
    
    # Create client
    client = LlamaStackClient(base_url=base_url)
    print("✅ Client created")
    
    print(f"🔍 Searching for: '{search_query}' in space '{space_key}'")
    
    try:
        # Use the exact working pattern from the example
        result = client.tool_runtime.invoke_tool(
            tool_name="confluence_search",
            kwargs={
                "query": search_query,
                "limit": 10,
                "spaces_filter": space_key
            }
        )
        print("✅ Confluence search successful!")
        print(f"Result: {result}")
        return {
            "success": True,
            "result": result,
            "search_query": search_query,
            "space_key": space_key
        }
    except Exception as e:
        print(f"❌ Confluence search failed: {e}")
        return {
            "success": False,
            "error": str(e),
            "search_query": search_query,
            "space_key": space_key
        }

print("✅ Simple Confluence search function 'search_confluence_simple' defined!")
print("   Uses the exact working pattern from simple-tool-test-working.ipynb")


✅ Simple Confluence search function 'search_confluence_simple' defined!
   Uses the exact working pattern from simple-tool-test-working.ipynb


In [21]:
# Test the simple working approach
print("🧪 Testing Simple Confluence search (exact working pattern)")
print("="*60)

# Test simple approach
simple_result = search_confluence_simple("OpenShift", space_key="ocp")

print(f"\n📋 SIMPLE APPROACH RESULTS:")
print(f"Success: {simple_result['success']}")

if simple_result['success']:
    print(f"Search query: {simple_result.get('search_query', 'Unknown')}")
    print(f"Space key: {simple_result.get('space_key', 'Unknown')}")
    print(f"\n📝 Search results:")
    print(f"{simple_result.get('result', 'No results')}")
else:
    print(f"❌ Error: {simple_result.get('error', 'Unknown error')}")

print("\n" + "="*60)


🧪 Testing Simple Confluence search (exact working pattern)
🔗 Connecting to: http://localhost:8321
✅ Client created
🔍 Searching for: 'OpenShift' in space 'ocp'
✅ Confluence search successful!
Result: ToolInvocationResult(content=[TextContentItem(text='[\n  {\n    "id": "262334",\n    "title": "The Pod Restarting Due to Out of Memory Error",\n    "type": "page",\n    "created": "",\n    "updated": "",\n    "url": "https://sukantadash.atlassian.net/wiki/spaces/ocp/pages/262334",\n    "space": {\n      "key": "ocp",\n      "name": "Space ocp"\n    },\n    "attachments": [],\n    "content": {\n      "value": "terminated by @@@hl@@@OpenShift@@@endhl@@@ with an OOMKilled (Out of Memory) error. This means the application was attempting to use more memory than the 512Mi limit allocated to it in its... configuration in @@@hl@@@OpenShift@@@endhl@@@ was increased from 512Mi to 1Gi. This provided the application with sufficient memory to operate correctly, and the pods have been stable since the ch